In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots
from pyamg.classical import ruge_stuben_solver

In [ ]:
from topo import SQuad
from poly import lagrange_list
from basis import LagrangeBasisQuad
from mesh import Mesh2D, uniform_nodes_2d
from assemble import simple_assembly, simple_build_rhs
from poisson import poisson_2d_Kloc

In [ ]:
L = 2*np.pi
x_max = L
y_max = L

k1 = 2.0
k2 = 2.0

In [ ]:
def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,2))
    x = X[:,0]
    y = X[:,1]
    sol  = np.sin(k1*2*np.pi*x/x_max)
    sol *= np.sin(k2*2*np.pi*y/y_max)
    return sol.reshape(shape)

def f2(X):
    sol  = -f(X)
    sol *=  (k1*2*np.pi/x_max)**2\
           +(k2*2*np.pi/y_max)**2
    return sol

In [ ]:
topo = SQuad()
errs = {}
order_vals = [1,2,3]
for order in order_vals:
    basis = LagrangeBasisQuad(topo, order)
    errs[order] = []
    for n_elems in [16, 32]:
        
        vertices, elem_to_vertex, boundary_vertices = \
                 uniform_nodes_2d(n_elems, x_max, y_max)
        mesh = Mesh2D(topo, basis)
        mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

        nodes = vertices[elem_to_vertex]
        jacb  = topo.calc_jacb(nodes)
        jacb_det = topo.calc_jacb_det(jacb)
        jacb_inv = topo.calc_jacb_inv(jacb)
        jacb = jacb[0]
        jacb_det = jacb_det[0]
        jacb_inv = jacb_inv[0]
 
        Kloc = poisson_2d_Kloc(basis, jacb_det, jacb_inv)
        K = simple_assembly(mesh, Kloc)
        
        rhs = simple_build_rhs(topo, basis, mesh, f2)

        ml = ruge_stuben_solver(K)
        residuals = []
        sol = ml.solve(rhs, tol=1e-12, 
                       residuals=residuals, maxiter=5000)
        
        X = vertices[:,0].reshape((n_elems+1, n_elems+1))
        Y = vertices[:,1].reshape((n_elems+1, n_elems+1))
        Z1 = sol[mesh.vertex_to_dof].ravel()
        Z2 = f(mesh.vertices).ravel()
        err = np.max(np.abs(Z1-Z2))
        
        errs[order].append(err)

In [ ]:
for order in order_vals:
    print order, -np.log2(errs[order][1]/errs[order][0])